In [7]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import numpy as np
from mesa.datacollection import DataCollector
import random


In [46]:
class CorruptionAgent(Agent):
    """ An agent with risk aversion $b$, capital endowment $k$"""
    def __init__(self, unique_id, model, parent=None):
        super().__init__(unique_id, model)
        self.steps = 0 
    
    def get_r(self):
        self.r = random.randint(0,10)
        
    def get_f(self):
        self.f = random.randint(10, 20)
        
    def step(self):
        self.get_r()
        self.get_f()

In [47]:
class CorruptionModel(Model):
    """A model with 1000 agents."""
    def __init__(self, population=10):
        """Create the model with the following parameters:
        Average level of risk aversion = b_bar
        Range of risk aversion = b_range
        Proportion of income spent on vigilance = gamma
        Mean human capital endowment in first generation = k_bar
        Equality in access to human capital = theta
        Initial value of social corruption = q_start
        Population = population
        Number of generations = generations
        Range of human capital endowment: k_range"""
        #Set parameters
        self.num_agents= population
        self.schedule = RandomActivation(self) 
        for i in range(self.num_agents):
            a = CorruptionAgent(i, self)
            self.schedule.add(a)       
            
        
        
    def step(self):
        ''' Advance the model by one step.'''
        for agent in self.schedule.agents:
            #getattr(agent, "get_r")() 
            agent.step()

In [48]:
model=CorruptionModel()

In [49]:
model.step()

In [50]:
[a.f for a in model.schedule.agents]

[11, 19, 17, 18, 10, 15, 13, 17, 16, 20]